# Homework 1

When not otherwise specified, use the following parameter values in experiment runs:
- `nr_clients` (N): 100
- `lr`: 0.01
- `client_fraction`(C): 0.1
- `nr_local_epochs` (E): 1
- `batch_size` (B): 100
- `nr_rounds`: 10
- `iid`: True

For all exercises, pass `seed = 10` to calls for splitting data, server initialization, or plotting.

In [1]:
import pandas as pd
import seaborn as sns
from base import *

n = 100
lr = 0.01
c = 0.1
e = 1
b = 100
nr_rounds = 10
iid = True
seed = 10

client_subsets = split(nr_clients=n, iid=iid, seed=seed)

## Exercise 1 - FedSGD with weights (3 points)client_subsets

### Question

_(2 points)_ Implement a version of FedSGD that uses weights in its updates, like FedAvg, instead of the gradients from the version of the tutorials. The two FedSGD versions should have the same test accuracy after each round (with a tolerance of at most 0.02%). To show this, compare their output for the following two scenarios over *5 rounds*:
- `lr = 0.01, client_subsets = split(100, True, ...), client_fraction = 0.5`
- `lr = 0.1, client_subsets = split(50, False, ...), client_fraction = 0.2`

*Tip:* You can use the existing FedAvg implementation to minimize the amount of code writing required.

_(1 point)_ Explain in which cases (about the different parameters for decentralized learning) the two are equivalent.

### Answer

In [2]:
client_subsets_1_a = split(n=100, iid=True, seed=seed)
client_subsets_1_b = split(n=50, iid=False, seed=seed)

fedsgd_gradient_df_a = FedSgdGradientServer(lr=0.01, client_subsets=client_subsets_1_a, client_fraction=0.5, seed=seed).run(nr_rounds=5).as_df()
fedavg_df_a = FedAvgServer(lr=0.01, batch_size=b, client_subsets=client_subsets_1_a, client_fraction=0.5, nr_local_epochs=e, seed=seed).run(nr_rounds=5).as_df()

fedsgd_gradient_df_b = FedSgdGradientServer(lr=0.1, client_subsets=client_subsets_1_b, client_fraction=0.2, seed=seed).run(nr_rounds=5).as_df()
fedavg_df_b = FedAvgServer(lr=0.1, batch_size=b, client_subsets=client_subsets_1_b, client_fraction=0.2, nr_local_epochs=e, seed=seed).run(nr_rounds=5).as_df()

merged_df_1 = pd.concat([fedsgd_gradient_df_a, fedavg_df_a, fedsgd_gradient_df_b, fedavg_df_b], ignore_index=True)

merged_df_1

'\nsample_split = split(100, True, seed)\nfedsgd_gradient_server = FedSgdGradientServer(0.01, sample_split, 0.5, seed)\nresult_fedsgd_gradient = fedsgd_gradient_server.run(5)\nfedsgd_gradient_df = result_fedsgd_gradient.as_df()\nfedsgd_gradient_df\n\nsample_split = split(50, False, seed)\nfedsgd_gradient_server = FedSgdGradientServer(0.1, sample_split, 0.2, seed)\nresult_fedsgd_gradient = fedsgd_gradient_server.run(5)\nfedsgd_gradient_df = result_fedsgd_gradient.as_df()\nfedsgd_gradient_df\n'

TODO

## Exercise 2 - Client number & fraction (4 points)

### Question

_(2 points)_ Run the necessary experiments to fill in the following table showing the final message count and test accuracy of FedSGD and FedAvg for different total client numbers:

| Algorithm | N   | C   | Message count | Test accuracy |
| --------- | --- | --- | ------------- | ------------- |
| FedSGD    | 10  | 0.1 |               |               |
| FedAvg    | 10  | 0.1 |               |               |
| FedSGD    | 50  | 0.1 |               |               |
| FedAvg    | 50  | 0.1 |               |               |
| FedSGD    | 100 | 0.1 |               |               |
| FedAvg    | 100 | 0.1 |               |               |

Is the relationship between the metrics and client numbers monotonous?

_(2 points)_ Run the experiments to fill in the table when varying the fraction of clients used in every round:

| Algorithm | N   | C    | Message count | Test accuracy |
| --------- | --- | ---- | ------------- | ------------- |
| FedSGD    | 100 | 0.01 |               |               |
| FedAvg    | 100 | 0.01 |               |               |
| FedSGD    | 100 | 0.1  |               |               |
| FedAvg    | 100 | 0.1  |               |               |
| FedSGD    | 100 | 0.2  |               |               |
| FedAvg    | 100 | 0.2  |               |               |

How does the observed pattern differ?

### Answer

In [3]:
client_subsets_2_a = [split(n, iid, seed) for n in [10,50,100]]

fedsgd_gradient_df = [FedSgdGradientServer(lr=lr, client_subsets=client_subsets_2_a[i], client_fraction=c, seed=seed).run(nr_rounds).as_df() for i in range (3)]
fedavg_df = [FedAvgServer(lr=lr, batch_size=b, client_subsets=client_subsets_2_a[i], client_fraction=c, nr_local_epochs=e, seed=seed).run(nr_rounds).as_df() for i in range (3)]

merged_df_2_a = pd.concat(fedsgd_gradient_df + fedavg_df, ignore_index=True)

In [4]:
merged_df_2_a

,Round,Algorithm,N,C,B,E,η,Seed,Message count,Test accuracy
0,1,FedSGDGradient,10,0.1,∞,1,0.01,10,2,9.11
1,2,FedSGDGradient,10,0.1,∞,1,0.01,10,4,13.95
2,1,FedSGDGradient,50,0.1,∞,1,0.01,10,10,9.13
3,2,FedSGDGradient,50,0.1,∞,1,0.01,10,20,13.47
4,1,FedSGDGradient,100,0.1,∞,1,0.01,10,20,9.01
5,2,FedSGDGradient,100,0.1,∞,1,0.01,10,40,13.83
6,1,FedAvg,10,0.1,100,1,0.01,10,2,74.89
7,2,FedAvg,10,0.1,100,1,0.01,10,4,86.80
8,1,FedAvg,50,0.1,100,1,0.01,10,10,47.54
9,2,FedAvg,50,0.1,100,1,0.01,10,20,64.91


TODO

In [5]:
fraction_of_clients = [0.01, 0.1, 0.2]

fedsgd_gradient_df = [FedSgdGradientServer(lr=lr, client_subsets=client_subsets, client_fraction=fraction_of_clients[i], seed=seed).run(nr_rounds).as_df() for i in range (3)]
fedavg_df = [FedAvgServer(lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=fraction_of_clients[i], nr_local_epochs=e, seed=seed).run(nr_rounds).as_df() for i in range (3)]

merged_df_2_b = pd.concat(fedsgd_gradient_df + fedavg_df, ignore_index=True)

In [6]:
merged_df_2_b

,Round,Algorithm,N,C,B,E,η,Seed,Message count,Test accuracy
0,1,FedSGDGradient,100,0.01,∞,1,0.01,10,2,9.40
1,2,FedSGDGradient,100,0.01,∞,1,0.01,10,4,14.52
2,1,FedSGDGradient,100,0.10,∞,1,0.01,10,20,9.01
3,2,FedSGDGradient,100,0.10,∞,1,0.01,10,40,13.83
4,1,FedSGDGradient,100,0.20,∞,1,0.01,10,40,8.89
5,2,FedSGDGradient,100,0.20,∞,1,0.01,10,80,13.74
6,1,FedAvg,100,0.01,100,1,0.01,10,2,30.20
7,2,FedAvg,100,0.01,100,1,0.01,10,4,41.31
8,1,FedAvg,100,0.10,100,1,0.01,10,20,30.55
9,2,FedAvg,100,0.10,100,1,0.01,10,40,46.23


TODO

## Exercise 3 - Batch size & learning rate (4 points)

### Question

_(2 points)_ Complete the following table with the last-round wall time and test accuracy of FedAvg under different batch sizes:

| Algorithm | B   | η    | Wall time | Test accuracy |
| --------- | --- | ---- | --------- | ------------- |
| FedAvg    | 10  | 0.01 |           |               |
| FedAvg    | 100 | 0.01 |           |               |
| FedAvg    | 500 | 0.01 |           |               |

What is the trade-off for larger batch sizes?

*Note:* The exact measured time will differ from run to run, even on the same machine, but the trends should remain the same. Furthermore, the effect of batch size on wall time is less pronounced and, depending on the exact hardware is sometimes minimal when running on CPU instead of GPU.

_(2 points)_ Add the corresponding values for varying learning rates, also for FedSGD, in the table below:

| Algorithm | B   | η     | Wall time | Test accuracy |
| --------- | --- | ----- | --------- | ------------- |
| FedSGD    | ∞   | 0.001 |           |               |
| FedAvg    | 100 | 0.001 |           |               |
| FedSGD    | ∞   | 0.01  |           |               |
| FedAvg    | 100 | 0.01  |           |               |
| FedSGD    | ∞   | 1.0   |           |               |
| FedAvg    | 100 | 1.0   |           |               |

Does the learning rate influence the wall time? For accuracy, how does it change the value obtained within the set number of rounds? Can it impact the model's convergence?

### Answer

In [7]:
batch_sizes = [10, 100, 500]

fedavg_df_3_a = [FedAvgServer(lr=lr, batch_size=batch_sizes[i], client_subsets=client_subsets, client_fraction=c, nr_local_epochs=e, seed=seed).run(nr_rounds).as_df(skip_wtime=False) for i in range (3)]

fedavg_df_3_a

[   Round Algorithm    N    C   B  E     η  Seed  Wall time  Message count  \
 0      1    FedAvg  100  0.1  10  1  0.01    10       29.3             20   
 1      2    FedAvg  100  0.1  10  1  0.01    10       79.0             40   
 
    Test accuracy  
 0          70.90  
 1          85.11  ,
    Round Algorithm    N    C    B  E     η  Seed  Wall time  Message count  \
 0      1    FedAvg  100  0.1  100  1  0.01    10       37.7             20   
 1      2    FedAvg  100  0.1  100  1  0.01    10       98.3             40   
 
    Test accuracy  
 0          30.55  
 1          46.23  ,
    Round Algorithm    N    C    B  E     η  Seed  Wall time  Message count  \
 0      1    FedAvg  100  0.1  500  1  0.01    10       59.2             20   
 1      2    FedAvg  100  0.1  500  1  0.01    10      146.9             40   
 
    Test accuracy  
 0          14.95  
 1          24.06  ]

TODO

In [ ]:
learning_rates = [0.001, 0.01, 1.0]

fedsgd_gradient_df = [FedSgdGradientServer(lr=learning_rates[i], client_subsets=client_subsets, client_fraction=c, seed=seed).run(nr_rounds).as_df(skip_wtime=False) for i in range (3)]
fedavg_df = [FedAvgServer(lr=learning_rates[i], batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=e, seed=seed).run(nr_rounds).as_df(skip_wtime=False) for i in range (3)]

merged_df_3_b = pd.concat(fedsgd_gradient_df + fedavg_df, ignore_index=True)

merged_df_3_b

Rounds:   0%|          | 0/2 [00:00<?, ?it/s]

TODO

## Exercise 4 - Local epoch count & (non-)IID data (5 points)

### Question

_(1 point)_ Create a line plot of the accuracy after each round for the following algorithm variants:

- FedSGD
- FedAvg (E=1)
- FedAvg (E=2)
- FedAvg (E=4)

How does FedAvg compare to FedSGD? What is the effect of increasing the work clients perform locally for each update in FedAvg?

_(2 points)_ Make one line plot of FedSGD and FedAvg under an IID and non-IID split for 15 rounds (leaving all other parameter values as they previously mentioned default). How does the non-IID setting affect the accuracy achieved by the two algorithms? What is the difference in terms of the smoothness of learning?

_(2 points)_ Make another plot for only non-IID splits, including the FedSGD and FedAvg configs from before, and add a version for each with a learning rate of 0.001 and client fraction of 0.5. How does the stability of the new variants compare to the old ones? Why do the changes in parameters have the observed effect?

### Answer

In [ ]:
local_epochs = [1, 2, 4]

fedsgd_gradient_df = FedSgdGradientServer(lr=lr, client_subsets=client_subsets, client_fraction=c, seed=seed).run(nr_rounds).as_df()
fedavg_df = [FedAvgServer(lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=local_epochs[i], seed=seed).run(nr_rounds).as_df() for i in range (3)]

df = pd.concat([fedavg_df, fedsgd_gradient_df], ignore_index=True)
ax = sns.lineplot(df, x="Round", y="Test accuracy", hue="Algorithm", seed=0)
_ = ax.set_xticks(df["Round"].unique())

TODO

In [ ]:
client_subsets_4_b = [split(n, iid, seed) for iid in [True, False]]

fedsgd_gradient_df_iid = FedSgdGradientServer(lr=lr, client_subsets=client_subsets_4_b[0], client_fraction=c, seed=seed).run(nr_rounds=15).as_df()
fedavg_df_iid = FedAvgServer(lr=lr, batch_size=b, client_subsets=client_subsets_4_b[0], client_fraction=c, nr_local_epochs=e, seed=seed).run(nr_rounds=15).as_df()

fedsgd_gradient_df = FedSgdGradientServer(lr=lr, client_subsets=client_subsets_4_b[1], client_fraction=c, seed=seed).run(nr_rounds=15).as_df()
fedavg_df = FedAvgServer(lr=lr, batch_size=b, client_subsets=client_subsets_4_b[1], client_fraction=c, nr_local_epochs=e, seed=seed).run(nr_rounds=15).as_df()

merged_df_4_b = pd.concat([fedsgd_gradient_df_iid, fedsgd_gradient_df, fedavg_df_iid, fedavg_df], ignore_index=True)
ax = sns.lineplot(df, x="Round", y="Test accuracy", hue="Algorithm", seed=0)
_ = ax.set_xticks(df["Round"].unique())

TODO

In [ ]:
fedsgd_gradient_df_4_c = FedSgdGradientServer(lr=0.001, client_subsets=client_subsets_4_b[1], client_fraction=0.5, seed=seed).run(nr_rounds=15).as_df()
fedavg_df_4_c = FedAvgServer(lr=0.001, batch_size=b, client_subsets=client_subsets_4_b[1], client_fraction=0.5, nr_local_epochs=e, seed=seed).run(nr_rounds=15).as_df()

merged_df_4_c = pd.concat([fedsgd_gradient, fedsgd_gradient_df_4_c, fedavg_df, fedavg_df_4_c], ignore_index=True)
ax = sns.lineplot(df, x="Round", y="Test accuracy", hue="Algorithm", seed=0)
_ = ax.set_xticks(df["Round"].unique())

TODO